In [1]:
!pip install konlpy

     |                                | 30 kB 1.5 MB/s eta 0:00:13

     |                                | 40 kB 1.4 MB/s eta 0:00:14

     |▏                               | 102 kB 910 kB/s eta 0:00:22

     |▏                               | 133 kB 910 kB/s eta 0:00:22

     |▎                               | 194 kB 910 kB/s eta 0:00:22

...

     |███████████████████████████████▊| 450 kB 24.1 MB/s eta 0:00:01

     |████████████████████████████████| 453 kB 24.1 MB/s 


  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.9.0
    Uninstalling beautifulsoup4-4.9.0:


      Successfully uninstalled beautifulsoup4-4.9.0


You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import sklearn
from sklearn.decomposition import TruncatedSVD
import seaborn as sns
from pandas_profiling import ProfileReport 

import pandas as pd
import re
import pickle
from gensim.models import word2vec
from gensim.models import FastText
from konlpy.tag import Mecab, Kkma, Okt, Komoran



In [3]:
train_df = pd.read_csv('../input/datahousewarming/data-house-warming-with-all-proudct-reviews.csv', sep='\t', encoding="UTF-16")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (21,22,23,24,25,26,27,28,29) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
train_df.sample(frac=0.4) 

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,name,review_count,review_average,scrap_count,view_count,review_user_id,...,house_expertise,house_color_list,house_style_list,house_constructions,house_family_list,house_like_count,house_reply_count,house_scrap_count,house_view_count,house_share_count
393746,78568,18670,NaN,312751,슈크림 세미워셔 차렵이불(세트) 10colors,1188,4.47,22358,74681,1117907.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18638,93572,33674,NaN,388715,순수원목 A사이드테이블 3colors,28574,4.53,20610,162635,6284378.0,...,홈스타일링,"베이지,라이트 브라운",내추럴,NaN,['싱글라이프'],264.0,59.0,750.0,25396.0,37.0
71094,93689,33791,NaN,388715,순수원목 A사이드테이블 3colors,28574,4.53,20610,162635,1360940.0,...,홈스타일링,"화이트,베이지,라이트 브라운,브라운,그린","미니멀&심플,내추럴",NaN,['싱글라이프'],418.0,77.0,1379.0,46553.0,59.0
279997,56606,56606,56606.0,309648,[주말특가] 스테이 프리미엄 차렵이불(세트) 10colors,8391,4.51,22294,395346,8154649.0,...,홈스타일링,"화이트,베이지,브라운,그린",내추럴,"원목마루,주방리모델링","신혼부부, 아기가 있는 집",344.0,32.0,854.0,48281.0,123.0
399748,84570,24672,NaN,405928,FW 화이트 원형테이블 1000size,619,4.59,7160,66396,852754.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163863,37338,37338,37338.0,51232,까페 2인 패브릭소파 7colors,386,4.57,13983,223705,3208366.0,...,홈스타일링,"화이트,브라운,그린",NaN,NaN,'싱글라이프',218.0,42.0,627.0,28627.0,48.0
383871,68693,8795,NaN,150960,타블도트 슈크레 수저통 3종 택1,1275,4.68,6626,70928,864508.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
322731,7553,7553,7553.0,52579,[쿠폰할인] 루비 2인소파 11colors,2730,4.58,69680,878488,6406705.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
354268,39090,39090,39090.0,86203,[쿠폰할인] 에이미 3인용 스타일클린 워셔블 패브릭소파,195,4.55,8945,151403,3235954.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413662 entries, 0 to 413661
Data columns (total 35 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Unnamed: 0                    413662 non-null  int64  
 1   Unnamed: 0.1                  413662 non-null  int64  
 2   Unnamed: 0.1.1                179234 non-null  float64
 3   id                            413662 non-null  int64  
 4   name                          413662 non-null  object 
 5   review_count                  413662 non-null  int64  
 6   review_average                413662 non-null  float64
 7   scrap_count                   413662 non-null  int64  
 8   view_count                    413662 non-null  int64  
 9   review_user_id                410708 non-null  float64
 10  review_status                 410708 non-null  object 
 11  review_text                   410706 non-null  object 
 12  review_star_durability        410708 non-nul

In [6]:
train_df2 = train_df.dropna(axis=0, subset = ['review_user_id', 'review_status', 'review_text', 'review_star_durability', 'review_star_design', 
                                   'review_star_cost', 'review_star_delivery', 'review_star_avg', 'product_info_id', 
                                   'production_info_name', 'production_info_brand_name', 'production_info_is_purchased'])

In [7]:
train_df2.isnull().sum()

Unnamed: 0                           0
Unnamed: 0.1                         0
Unnamed: 0.1.1                  233495
id                                   0
name                                 0
review_count                         0
review_average                       0
scrap_count                          0
view_count                           0
review_user_id                       0
review_status                        0
review_text                          0
review_star_durability               0
review_star_design                   0
review_star_cost                     0
review_star_delivery                 0
review_star_avg                      0
product_info_id                      0
production_info_name                 0
production_info_brand_name           0
production_info_explain          10489
production_info_is_purchased         0
house_residence                  96288
house_area                       96288
house_region                    166224
house_expertise          

In [8]:
train_df2 = train_df2.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'], axis=1)

In [9]:
train_df2.profile_report()

In [ ]:
# 형태소 분석기 선언

kkma = Kkma()


In [ ]:
# 전처리 함수 선언
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

def remove_brackets(text):
    return re.sub("[\(\[].*?[\)\]]", "", text)

def remove_punc(text):
    return re.sub("[!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~]", ' ', text)

def keep_korean(text):
    return re.sub(r"[^가-힣]", ' ', text)

def pos(text):
    return [token for token, tag in kkma.pos(text) if tag[:2] == 'NN' or tag[:2] == 'NP' or tag[:2] == 'SL']


In [ ]:
reviews = train_df2['review_text']

In [ ]:
reviews

In [ ]:
review_token_list

In [ ]:
review_token_list = [sentence.split(' ') for sentence in reviews]

In [ ]:
string_review_token_list = list()

for review_tokens in review_token_list:
    for token in review_tokens:
        string_review_tokens.append(str(token.strip()))
        string_review_token_list.append(string_review_tokens)

In [ ]:
string_review_token_list

In [ ]:
string_review_token_list

In [ ]:
review = [remove_brackets(string_review_token_list) for s in string_review_token_list]

In [ ]:
review = remove_tags(review)

In [ ]:
review = remove_punc(review)

In [ ]:
review = keep_korean(review)

In [ ]:
review = pos(review)

In [ ]:
newReview = [kkma.nouns(s) for s in review]

In [ ]:
newReview

In [ ]:
# 옵션 설정
num_features = 300  # 문자 벡터 차원 수
min_word_count = 3  # 최소 문자 수
num_workers = 4  # 병렬 처리 스레드 수
context = 5  # 문자열 창 크기
# downsampling = 1e-3  # 문자 빈도수 Downsample



In [ ]:
# word2vec 모델 학습
model = word2vec.Word2Vec(newReview,
                          workers=num_workers,
                          size=num_features,
                          min_count=min_word_count,
                          window=context, sg=0)


In [ ]:
model.wv.vectors.shape

In [ ]:
from sklearn.manifold import TSNE 
from sklearn.decomposition import PCA 
import matplotlib as mpl 
import matplotlib.pyplot as plt 
import pandas as pd 
from gensim.models import KeyedVectors



In [ ]:
def show_pca(): # PCA 모델을 생성합니다 
    pca = PCA(n_components=2) 
    pca.fit(X_show) # 처음 두 개의 주성분으로 숫자 데이터를 변환합니다 
    x_pca = pca.transform(X_show) 
    
    plt.figure(figsize=(30, 20)) 
    plt.xlim(x_pca[:, 0].min(), x_pca[:, 0].max()) 
    plt.ylim(x_pca[:, 1].min(), x_pca[:, 1].max()) 
    for i in range(len(X_show)): 
        plt.text(x_pca[i, 0], x_pca[i, 1], str(vocab_show[i]), 
                 fontdict={'weight': 'bold', 'size': 9}) 
        plt.xlabel("ㅁ") 
        plt.ylabel("ㅁ") 
        plt.show()



In [ ]:
vocab = list(model.wv.vocab) 
X = model[vocab]

sz = 100 
X_show = X[:sz,:] 
vocab_show = vocab[:sz]

show_pca()